In [2]:
import geopandas
import ee
import geemap.foliumap as geemap
import geemap as gee

import numpy as np
import matplotlib.pyplot as plt
import folium
import requests
import rasterio
from rasterio.plot import show

In [9]:
def extract_pixel_value(img, region, bandName, scale):
    """Extract pixel value from an image for a given region"""
    value = img.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=region,
        scale=scale,
        maxPixels=1e9
    ).get(bandName)
    return value


# ##### Define a method for displaying Earth Engine image tiles to folium map
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    layer = folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    )
    layer.add_to(self)
    return layer

In [17]:
def extract_gee_data(spec_file_path: str, data):
    # découper chaque géométrie en sous géométrie ayant une valeur unique fournie par GEE
    gfc = ee.Image("UMD/hansen/global_forest_change_2021_v1_9")

    output = []
    for input_zone in data:
        image = gfc.clip(input_zone)
        bands_list = image.getInfo()['bands']
        band = bands_list[0]
        width = band['dimensions'][0]
        height = band['dimensions'][1]
        treecover = image.select(band['id'])
        scale = image.projection().nominalScale().getInfo()

        reduction = treecover.reduceToVectors(
            geometry=None,
            crs=image.projection(),
            scale=scale,
            geometryType='polygon',
            eightConnected=True,
            labelProperty='zone',
            maxPixels=1e15,
            tileScale=16
        )
        output.append(input_zone)

    return output

    # OU

    # donner min, max, mean pour chaque géométrie donnée en entrée


ee.Initialize()
zones = [
    ee.Geometry.Polygon([[[166.40, -22.22], [166.40, -22.15], [166.42, -22.13], [166.45, -22.15], [166.45, -22.22]]]),
    ee.Geometry.Polygon([[[166.50, -22.10], [166.50, -22.15], [166.52, -22.12], [166.54, -22.15], [166.54, -22.10]]]),
]
map = folium.Map(location=[(-22.22 + -22.10) / 2, (166.40 + 166.45) / 2], zoom_start=12)
for zone in zones:
    folium.GeoJson(zone.toGeoJSON()).add_to(map)
display(map)

In [ ]:
gfc = ee.Image("UMD/hansen/global_forest_change_2021_v1_9")
image = gfc.clip(ma_zone)
bands_list = image.getInfo()['bands']
band = bands_list[0]
width = band['dimensions'][0]
height = band['dimensions'][1]
treecover = image.select(band['id'])
scale = image.projection().nominalScale().getInfo()

# https://developers.google.com/earth-engine/guides/reducers_reduce_to_vectors
vectors = treecover.reduceToVectors(
    geometry=None,
    crs=image.projection(),
    scale=scale,
    geometryType='polygon',
    eightConnected=True,
    labelProperty='zone',
    maxPixels=1e15,
    tileScale=16
).geometry()
result = ee.FeatureCollection([ee.Feature(vectors)])
map.add_ee_layer(result, {'min': 0.0, 'max': 100.0, 'palette': ['ffffff', 'ffbbbb', '0000ff']}, "toto")
display(map)

In [ ]:
zones = treecover.gt(30).add(treecover.gt(60))
# zones = zones.updateMask(zones.neq(0))
vectors = zones.addBands(treecover).reduceToVectors(
    geometry=None,
    crs=image.projection(),
    scale=scale,
    geometryType='polygon',
    eightConnected=False,
    labelProperty='zone',
    maxPixels=1e15,
    tileScale=16
)
map.add_ee_layer(zones, {'min': 0, 'max': 2, 'palette': ['ffffff', '449152', '01330a']}, 'treecover')

# Make a display image for the vectors, add it to the map.
# display = ee.Image(0).updateMask(0).paint(vectors, '000000', 3)
# map.add_ee_layer(ee.Image(0).updateMask(0).paint(vectors, '000000', 3), {'palette': '000000'}, 'vectors')
display(map)

In [6]:
# import des spécifications
import yaml
from yaml.loader import SafeLoader
import geopandas as gpd
from shapely.geometry import Polygon
import json

spec_file_path = './input_spec_example.yml'

with open(spec_file_path) as f:
    specs = yaml.load(f, Loader=SafeLoader)

# création du fond de carte
map = folium.Map(location=[(-22.2276038 + -21.99) / 2, (166.606135574 + 166.387779807) / 2], zoom_start=12)

# création de l'input : liste de geodataframe
# ma_zone_ee = ee.Geometry.Polygon([[[166.387779807,-22.2276038],[166.387779807,-22.039679148],[166.50,-21.99],[166.606135574,-22.039679148],[166.606135574,-22.2276038]]])
coords = [(166.387779807, -22.2276038), (166.387779807, -22.039679148), (166.50, -21.99),
          (166.606135574, -22.039679148), (166.606135574, -22.2276038)]
ma_zone = Polygon(coords)
gdf = gpd.GeoDataFrame(crs=specs['epsg'], geometry=[ma_zone])
input = gdf

# affichage de la zone d'entrée sur la carte
folium.GeoJson(gdf.to_json()).add_to(map)

# création de la variable de sortie
output_polygons = []

for geometry in input['geometry']:
    xx, yy = geometry.exterior.coords.xy
    coords = np.dstack((xx, yy)).tolist()
    ee_zone = ee.Geometry.Polygon(coords)
    image_zonee = ee.Image(specs['confRaster']['uri_image']).select(specs['keepList']).clip(ee_zone)

    scale = image_zonee.projection().nominalScale().getInfo()
    # vectors = image_zonee.addBands(srcImg=image_zonee).reduceToVectors(
    vectors = image_zonee.addBands(srcImg=image_zonee).reduceToVectors(
        geometry=None,
        crs=specs['epsg'],
        scale=scale,
        geometryType='polygon',
        eightConnected=False,
        labelProperty='zone',
        maxPixels=1e15,
        tileScale=16,
        reducer=ee.Reducer.mean()
    )
    vectorsSize = vectors.size()
    vectorsList = vectors.toList(vectorsSize)
    mapid = vectors.getMapId()
    folium.TileLayer(
        tiles=mapid['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        overlay=True,
        name='border',
    ).add_to(map)
    # for vector in vectorsList:
    #     print(type(vectorsList.get(5059)))
    # print(ee.Geometry(vectorsList.get(5059)).getInfo()['geometry']['coordinates'])
    for idx in range(vectorsSize.getInfo()):
        if idx == 10:
            break
    sub_polygon = Polygon(ee.Geometry(vectorsList.get(idx)).getInfo()['geometry']['coordinates'][0])
    #     output_polygons.append(sub_polygon)

output = gpd.GeoDataFrame(crs=specs['epsg'], geometry=output_polygons)
# print(output[8])
# print(output)

display(map)

In [267]:
output

,geometry
0,"POLYGON ((166.38775 -22.04200, 166.38800 -22.0..."
1,"POLYGON ((166.38775 -22.04550, 166.38800 -22.0..."
2,"POLYGON ((166.38775 -22.04575, 166.38800 -22.0..."
3,"POLYGON ((166.38775 -22.04600, 166.38800 -22.0..."
4,"POLYGON ((166.38775 -22.04800, 166.38800 -22.0..."
5,"POLYGON ((166.38775 -22.04875, 166.38800 -22.0..."
6,"POLYGON ((166.38775 -22.04900, 166.38800 -22.0..."
7,"POLYGON ((166.38775 -22.05075, 166.38800 -22.0..."
8,"POLYGON ((166.38775 -22.05100, 166.38800 -22.0..."
9,"POLYGON ((166.38775 -22.05125, 166.38800 -22.0..."


In [59]:
# import des spécifications
import yaml
from yaml.loader import SafeLoader
import geopandas as gpd
import folium
import geemap.foliumap as geemap
import geemap as gee
import math
import affine
import rasterstats as rs
import ee
import matplotlib.pyplot as plt
from rasterio.plot import show

service_account = 'ee-oeil@surfor.iam.gserviceaccount.com'
credentials = ee.ServiceAccountCredentials(service_account, 'surfor-8383e43c3aa7.json')
ee.Initialize(credentials)


def round_up(x, increment):
    return math.ceil(x / increment) * increment

def round_down(x, increment):
    return math.floor(x / increment) * increment

spec_file_path = './input_spec_example.yml'

with open(spec_file_path) as f:
    specs = yaml.load(f, Loader=SafeLoader)

# création de l'input : geodataframe avec N polygone et N points
zone1 = ee.Geometry.Polygon(
        [[[166.40, -22.22], [166.40, -22.15], [166.42, -22.13], [166.45, -22.15], [166.45, -22.22]]])
zone2 = ee.Geometry.Polygon(
        [[[166.50, -22.10], [166.50, -22.15], [166.52, -22.12], [166.54, -22.15], [166.54, -22.10]]])
features = [
    ee.Feature(zone1, {'name': 'zone1'}),
    ee.Feature(zone2, {'name': 'zone2'}),
]
input_gdf = gee.ee_to_geopandas(ee.FeatureCollection(features))

# vérification du contenu de l'entrée
if len(input_gdf) == 0:
    raise 'Aucune donnée à traiter'
else
print(input_gdf.geom_type)

# image de référence
image = ee.Image(specs['confRaster']['uri_image']).select(specs['keepList'])
scale = image.getInfo()['bands'][0]['crs_transform'][0]
default_value = specs['confRaster']['defaultValue']
projection = specs['epsg']

# indiquer le crs si pas déjà fait
if input_gdf.crs is None:
    input_gdf = input_gdf.set_crs(crs=projection)

# traitement sur les polygones un à un pour éviter de dépasser la limite de GEE de 262144 pixels
feature_collection = geemap.geopandas_to_ee(input_gdf)
features_list = feature_collection.toList(feature_collection.size())
output_features = []
for idx in range (feature_collection.size().getInfo()):
    feature = ee.Feature(features_list.get(idx))
    # récupération du raster correspondant au polygon
    image_clipped = gee.ee_to_numpy(image, region=ee.FeatureCollection([feature]), default_value=default_value)
    image_clipped_reshaped = None
    if image_clipped is not None:
        image_clipped_reshaped = image_clipped.reshape(image_clipped.shape[0], (image_clipped.shape[1]*image_clipped.shape[2]))

    # récupération de l'affine
    sample_gdf = gee.ee_to_geopandas(ee.FeatureCollection([feature]))
    xmin, _, _, ymax = sample_gdf.total_bounds
    transform = affine.Affine.from_gdal(round_down(xmin, scale) , scale ,0, round_up(ymax, scale) ,0,-scale)

    # fig, ax = plt.subplots(figsize=(50, 12))
    # sample_gdf.plot(ax=ax, edgecolor='blue', alpha=0.5)
    # show(image_clipped_reshaped, ax=ax, transform=transform)
    # plt.show()

    # récupération des stats de la zone
    if image_clipped_reshaped is not None:
        stats = rs.zonal_stats(sample_gdf, image_clipped_reshaped, stats=['min', 'max', 'mean', 'median', 'std'],
                       affine=transform, nodata=default_value,geojson_out=True,raster_out=True, all_touched=True)
        output_features.append(stats[0])
output_gdf = gpd.GeoDataFrame.from_features(output_features, crs=projection)

0    Polygon
1    Polygon
dtype: object



KeyboardInterrupt



In [60]:
print(len(input_gdf))

2


In [48]:
image.getInfo()['bands'][0]['crs_transform'][0]

0.00025

In [26]:
scale = 0.0025
xmin, _, _, ymax = sample_gdf.total_bounds
transform = affine.Affine.from_gdal(round_down(xmin, scale) , scale ,0, round_up(ymax, scale) ,0,-scale)
transform

Affine(0.0025, 0.0, 166.5,
       0.0, -0.0025, -22.1)